In [14]:
import os
os.chdir("/root/workspace/code/midas/")
from os import path
from os.path import join as pj
import argparse
import sys
sys.path.append("modules")
import utils
import numpy as np
import anndata as ad
import pandas as pd
import copy
import re

In [15]:
parser = argparse.ArgumentParser()
parser.add_argument('--tasks', type=str, nargs='+',  default=["bm"])
parser.add_argument('--experiment', type=str, default='e0')
parser.add_argument('--model', type=str, default='la_1')
# parser.add_argument('--tasks', type=str, nargs='+',  default=["bm_transfer"])
# parser.add_argument('--experiment', type=str, default='no_map_ref')
# parser.add_argument('--model', type=str, default='default')
parser.add_argument('--methods', type=str, nargs='+', default=["midas_embed", "scmomat", "scvaeit", "stabmap", "multigrate"])
# parser.add_argument('--init_models', type=str, nargs='+', default=["sp_00001899"])
parser.add_argument('--init_models', type=str, nargs='+', default=["sp_00001299", "sp_00001399", "sp_00001499", "sp_00001599",
    "sp_00001699", "sp_00001799", "sp_00001899", "sp_00001999", "sp_00002099", "sp_00002199", "sp_00002299", "sp_00002399",
    "sp_00002499", "sp_00002599", "sp_00002699", "sp_00002799", "sp_00002899", "sp_00002999", "sp_00003099", "sp_00003199",
    "sp_00003299", "sp_00003399", "sp_00003499", "sp_00003599", "sp_00003699", "sp_00003799", "sp_00003899", "sp_00003999"])
o, _ = parser.parse_known_args()  # for python interactive
# o = parser.parse_args()

In [16]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--tasks', type=str, nargs='+',  default=["dogma_full"])
# parser.add_argument('--experiment', type=str, default='e0')
# parser.add_argument('--model', type=str, default='')
# parser.add_argument('--methods', type=str, nargs='+', default=["midas_embed"])
# parser.add_argument('--init_models', type=str, nargs='+', default=["sp_00000999", "sp_00001099", "sp_00001199", "sp_00001299", "sp_00001399", "sp_00001499", "sp_00001599",
#     "sp_00001699", "sp_00001799", "sp_00001899", "sp_00001999", ])
# o, _ = parser.parse_known_args()  # for python interactive
# # o = parser.parse_args()

In [17]:
# mosaic results
df_batch_bio_embed = []
for task in o.tasks:
    for method in o.methods:
        if method == "midas_embed":
            for init_model in o.init_models:
                fp = pj("result", "comparison", task, method, o.experiment, o.model, init_model, "metrics_batch_bio.xlsx")
                if path.exists(fp):
                    batch_bio_embed = pd.read_excel(fp)
                    batch_bio_embed["Task"] = task
                    batch_bio_embed["Method"] = method
                    batch_bio_embed["Init model"] = init_model
                    df_batch_bio_embed.append(batch_bio_embed)
        else:
            fp = pj("result", "comparison", task, method, "metrics_batch_bio.xlsx")
            if path.exists(fp):
                batch_bio_embed = pd.read_excel(fp)
                batch_bio_embed["Task"] = task
                batch_bio_embed["Method"] = method
                batch_bio_embed["Init model"] = "-"
                df_batch_bio_embed.append(batch_bio_embed)
df_batch_bio_embed_cat = pd.concat(df_batch_bio_embed, axis=0)
df_batch_bio_embed_cat

,iLISI,graph_conn,kBET,NMI,ARI,il_score_f1,cLISI,Task,Method,Init model
0,0.444038,0.977005,0.161627,0.701286,0.718068,0.778514,0.990181,bm,midas_embed,sp_00001299
0,0.441586,0.974864,0.168738,0.701209,0.719186,0.790743,0.990215,bm,midas_embed,sp_00001399
0,0.404708,0.977465,0.120668,0.701852,0.719456,0.785465,0.991658,bm,midas_embed,sp_00001499
0,0.433667,0.977138,0.160894,0.699969,0.716735,0.794337,0.990732,bm,midas_embed,sp_00001599
0,0.457440,0.975188,0.195038,0.701096,0.715066,0.785930,0.990323,bm,midas_embed,sp_00001699
0,0.450063,0.976708,0.166918,0.699761,0.716755,0.792448,0.990398,bm,midas_embed,sp_00001799
0,0.456167,0.978438,0.192231,0.697087,0.711041,0.780354,0.990693,bm,midas_embed,sp_00001899
0,0.445664,0.974870,0.166479,0.699723,0.716055,0.780067,0.989528,bm,midas_embed,sp_00001999
0,0.036365,0.956738,0.003493,0.546847,0.422546,0.629839,0.972562,bm,scmomat,-
0,0.001152,0.963598,0.006088,0.591891,0.490438,0.690401,0.995906,bm,scvaeit,-


In [18]:
df_cat = df_batch_bio_embed_cat
df_mean_cat = copy.deepcopy(df_cat)
df_mean_cat["batch_score"] = df_cat[["iLISI", "graph_conn", "kBET"]].mean(axis=1)
df_mean_cat["bio_score"] = df_cat[["NMI", "ARI", "il_score_f1", "cLISI"]].mean(axis=1)
df_mean_cat["overall_score"] = 0.4 * df_mean_cat["batch_score"] + 0.6 * df_mean_cat["bio_score"]
df_mean_cat = df_mean_cat[["Task", "Method", "Init model", "iLISI", "graph_conn", "kBET", "batch_score", "NMI", "ARI", "il_score_f1", "cLISI", "bio_score", "overall_score"]]
df_mean_cat_sorted = df_mean_cat.sort_values("overall_score", ascending=False, inplace=False)
df_mean_cat_sorted


,Task,Method,Init model,iLISI,graph_conn,kBET,batch_score,NMI,ARI,il_score_f1,cLISI,bio_score,overall_score
0,bm,midas_embed,sp_00001699,0.457440,0.975188,0.195038,0.542555,0.701096,0.715066,0.785930,0.990323,0.798104,0.695884
0,bm,midas_embed,sp_00001899,0.456167,0.978438,0.192231,0.542279,0.697087,0.711041,0.780354,0.990693,0.794794,0.693788
0,bm,midas_embed,sp_00001799,0.450063,0.976708,0.166918,0.531230,0.699761,0.716755,0.792448,0.990398,0.799840,0.692396
0,bm,midas_embed,sp_00001399,0.441586,0.974864,0.168738,0.528396,0.701209,0.719186,0.790743,0.990215,0.800338,0.691561
0,bm,midas_embed,sp_00001599,0.433667,0.977138,0.160894,0.523899,0.699969,0.716735,0.794337,0.990732,0.800443,0.689826
0,bm,midas_embed,sp_00001999,0.445664,0.974870,0.166479,0.529004,0.699723,0.716055,0.780067,0.989528,0.796343,0.689407
0,bm,midas_embed,sp_00001299,0.444038,0.977005,0.161627,0.527557,0.701286,0.718068,0.778514,0.990181,0.797012,0.689230
0,bm,midas_embed,sp_00001499,0.404708,0.977465,0.120668,0.500947,0.701852,0.719456,0.785465,0.991658,0.799608,0.680143
0,bm,multigrate,-,0.467413,0.977318,0.036829,0.493853,0.582422,0.543110,0.648941,0.923298,0.674443,0.602207
0,bm,scvaeit,-,0.001152,0.963598,0.006088,0.323613,0.591891,0.490438,0.690401,0.995906,0.692159,0.544740


In [19]:
# out_dir = pj("eval", "plot", "data")
# utils.mkdir(out_dir, remove_old=False)
# if o.mosaic == 0:
#     ms = "_sota_"
# elif o.sota == 0:
#     ms = "_mosaic_"
# else:
#     ms = "_sota+mosaic_"
# if o.method == "midas_embed":
#     df_mean_cat_sorted.to_excel(pj(out_dir, "scib_metrics"+ms+o.tasks[0].split("_")[0]+"_"+o.experiment+"_"+o.model+"_"+o.init_model+"_sorted.xlsx"))
#     df_mean_cat.to_excel(pj(out_dir, "scib_metrics"+ms+o.tasks[0].split("_")[0]+"_"+o.experiment+"_"+o.model+"_"+o.init_model+"_unsorted.xlsx"))
# else:
#     df_mean_cat_sorted.to_excel(pj(out_dir, "scib_metrics"+ms+o.tasks[0].split("_")[0]+"_"+o.method+"_sorted.xlsx"))
#     df_mean_cat.to_excel(pj(out_dir, "scib_metrics"+ms+o.tasks[0].split("_")[0]+"_"+o.method+"_unsorted.xlsx"))